In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/google-stock-price/Google_Stock_Price_Train.csv')
df_test = pd.read_csv('/kaggle/input/google-stock-price/Google_Stock_Price_Test.csv')

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(pd.to_datetime(df['Date']),df['Open'])

In [ ]:
df['Close']=df['Close'].apply(lambda X:(X.replace(',','')))



In [ ]:
df['Volume']=df['Volume'].apply(lambda X:float(X.replace(',','')))



In [ ]:
df['Close'] = pd.to_numeric(df['Close'],errors = 'coerce')
df['Volume'] = pd.to_numeric(df['Volume'],errors = 'coerce')


In [ ]:
df.info()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df[['Open','High','Low','Close']])
X_scaled = scaler.transform(df[['Open','High','Low','Close']])

In [ ]:
X=[]
y=[]
for i in range(0,len(X_scaled)-60):
    row = X_scaled[i:i+60, :]
    X.append(row)
    y.append(X_scaled[i+60,0])
X = np.array(X)
X.shape

In [ ]:
X.shape

In [ ]:
#X_train = X_train.reshape(-1, 60, 1)
#X_train.shape

In [ ]:
y=np.array(y)
y.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(60,4)))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))

In [ ]:
model.compile(loss='mean_squared_error',optimizer="Adam")

In [ ]:
hist = model.fit(X,y, batch_size=32, epochs=50)

In [ ]:
plt.plot(hist.history['loss'])

In [ ]:
# Testing the model
# X_scaled[-60:]
X_test_scaled=scaler.transform(df_test[['Open','High','Low','Close']])
X_test_scaled=np.vstack([X_scaled[-60:],X_test_scaled])
X_test_scaled.shape

In [ ]:
X=[]
for i in range(0,len(X_test_scaled)-60):
    row = X_test_scaled[i:i+60, :]
    X.append(row)
X = np.array(X)
X.shape

In [ ]:
y_pre = model.predict(X)

In [ ]:
dummy=np.zeros((20,3))
d=np.hstack([y_pre,dummy])

In [ ]:
d = scaler.inverse_transform(d)
d


In [ ]:
plt.plot(df_test['Open'], c='blue')
plt.plot(d[:,0], c='red')

In [ ]:
len(y_pre)